[Reference](https://github.com/meijieru/crnn.pytorch)

In [ ]:
import torch.nn as nn
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
from PIL import Image
import sys
import os
import torch.nn.init as weight_init

In [ ]:
sys.executable

In [ ]:
os.getcwd()

In [ ]:
os.chdir('./crnn.pytorch/')

In [ ]:
import random
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import numpy as np
from warpctc_pytorch import CTCLoss
import os
import utils
import dataset
import lmdb


import models.crnn as crnn

In [ ]:
import time
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from PIL import Image
import torch
from torch.optim import sgd
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
from PIL import Image

In [ ]:
class BidirectionalLSTM(nn.Module):
    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        for name, param in self.rnn.named_parameters():
            weight_init.normal(mean=0,std=1,tensor=param)
        self.embedding = nn.Linear(nHidden * 2, nOut)
        self.embedding.weight.data.normal_(mean=0,std=1)
    '''
    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)
        return output
    '''

In [ ]:
model = crnn.CRNN(32, 1, 37, 256)
model = model.cuda()
model.load_state_dict(torch.load("./data/crnn.pth"))

In [ ]:
df = pd.read_pickle('../../datamartV1.pkl')

In [ ]:
''''
for param in model.parameters():
    param.requires_grad = False
''''

In [ ]:
removed = list(model.rnn.children())[:-1]

In [ ]:
model.rnn = nn.Sequential(*removed)

In [ ]:
embedding = nn.Linear(256 * 2, 19)

In [ ]:
embedding.weight.data.normal_(mean=0,std=1)

In [ ]:
model.rnn.add_module("1",BidirectionalLSTM(256, 256, 19))

In [ ]:
model

In [ ]:
torch.save(model.state_dict(), './data/crnnV1.pkl')

In [ ]:
model = crnn.CRNN(32, 1, 19, 256)

In [ ]:
model.load_state_dict(torch.load('./data/crnnV1.pkl'))

In [ ]:
model

In [ ]:
X = np.array([i for i in df.image.values])

In [ ]:
X.shape

In [ ]:
X = X.reshape(4197,1, 100, 1000)

In [ ]:
X = torch.FloatTensor(X).type(torch.cuda.FloatTensor)

In [ ]:
env = lmdb.open("../TrainingDataset",max_readers=1,readonly=True,lock=False,readahead=False,meminit=False)

In [ ]:
variable = Variable(X, requires_grad=True)

In [ ]:
model = crnn.CRNN(32, 1, 10, 256)
model = model.cuda()
#(self, imgH, nc, nclass, nh, n_rnn=2, leakyRelu=False):

In [ ]:
model_path = './data/crnn.pth'

In [ ]:
variable.data.shape

In [ ]:
alphabet = '0123456789abcdefghijklmnopqrstuvwxyz'
converter = utils.strLabelConverter(alphabet)

In [ ]:
converter

In [ ]:
class strLabelConverter(object):
    """Convert between str and label.

    NOTE:
        Insert `blank` to the alphabet for CTC.

    Args:
        alphabet (str): set of the possible characters.
        ignore_case (bool, default=True): whether or not to ignore all of the case.
    """

    def __init__(self, alphabet, ignore_case=True):
        self._ignore_case = ignore_case
        if self._ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'  # for `-1` index

        self.dict = {}
        for i, char in enumerate(alphabet):
            # NOTE: 0 is reserved for 'blank' required by wrap_ctc
            self.dict[char] = i + 1

In [ ]:
pd.read_csv()

In [ ]:
transformer = dataset.resizeNormalize((100, 32))
image = Image.open(img_path).convert('L')
image = transformer(image)

In [ ]:
model.eval()
preds = model(X)

In [ ]:
_, preds = preds.max(2)
preds = preds.transpose(1, 0).contiguous().view(-1)

preds_size = Variable(torch.IntTensor([preds.size(0)]))
raw_pred = converter.decode(preds.data, preds_size.data, raw=True)
sim_pred = converter.decode(preds.data, preds_size.data, raw=False)
print('%-20s => %-20s' % (raw_pred, sim_pred))